# Demonstration of DCBC evaluation using GPU acceleration
This notebook shows an quick example of a Distance controlled boundary coefficient (DCBC) evaluation between numpy cpu vs. pytorch gpu version.

## Installation and Dependencies
Ensure Python version >= 3.6 and pip installable on your system.

`pip install nibabel scipy numpy sklearn matplotlib torch`

To use PyTorch GPU acceleration, please install the correct PyTorch and CUDA version. Detailed torch+cuda installation can be found in PyTorch official webpage at https://pytorch.org/

Below is a quick example of DCBC evaluation using CPU and GPU, and the speed comparison when evaluating `Choi 2012` 7 parcellation of striatum. In this example, we only focus on speed comparison between CPU and GPU version, the detailed step-by-step volumetric parcellation evaluation can be found elsewhere in notebook `example_volume.ipynb`.

## 1. Numpy CPU

In [1]:
import mat73, time
import numpy as np
import scipy as sp
import nibabel as nb
from utilities import compute_dist, plot_single
from dcbc import compute_DCBC

# Load mask voxel index
vol_ind = mat73.loadmat('../data/striatum_avrgDataStruct.mat')['volIndx']
vol_ind = vol_ind.astype(int)

In [2]:
# Load parcellation given the mask file or voxel indices
parcel_mni = nb.load('../parcellations/volume_striatum/masked_par_choi_7.nii.gz').get_fdata()
coord = np.unravel_index(vol_ind - 1, parcel_mni.shape, 'F')  # Note: the linear indexing in numpy is column-order
parcels = np.rint(parcel_mni[coord[0], coord[1], coord[2]])

In [3]:
# Compute the distance matrix between voxel pairs using the mask file, numpy default C-order
coord = np.asarray(coord).transpose()
dist = compute_dist(coord, 2, backend='numpy')
print(dist.shape)

(4135, 4135)


In [4]:
# Load functional profile (betas) and several parameters for evaluation settings
T = mat73.loadmat('../data/striatum_avrgDataStruct.mat')['T']
returnsubj = [2,3,4,6,8,9,10,12,14,15,17,18,19,20,21,22,24,25,26,27,28,29,30,31]
session, maxDist, binWidth = 1, 90, 5

Now, we start the real DCBC evaluation on the given parcellation using selected subjects and given experiment settings. So here we set the bin width = 5 mm, the maximum distance between any pair of voxels is 90 mm. We only use subjects session 1 data.

In [8]:
wcorr_array, bcorr_array, dcbc_array = np.array([]), np.array([]), np.array([])

tic = time.perf_counter()
for sub in returnsubj:
    data = T['data'][(T['SN'] == sub) & (T['sess'] == session)].T
    R = compute_DCBC(maxDist=maxDist, func=data, dist=dist, binWidth=binWidth, parcellation=parcels, backend='numpy')
    wcorr_array = np.append(wcorr_array, R['corr_within'])
    bcorr_array = np.append(bcorr_array, R['corr_between'])
    dcbc_array = np.append(dcbc_array, R['DCBC'])

toc = time.perf_counter()
print(f"Numpy CPU version DCBC calculation used {toc - tic:0.4f}s")
# print(wcorr_array, bcorr_array, dcbc_array)

Numpy CPU version DCBC calculation used 90.6093s


## 2. Torch GPU

In [5]:
import torch as pt
from utilities import convert_numpy_to_torch_sparse

DEVICE = 'cuda' if pt.cuda.is_available() else 'cpu'
pt.set_default_tensor_type(pt.cuda.FloatTensor
                           if pt.cuda.is_available() else
                           pt.FloatTensor)

# Covert input parcellation and distance from numpy to torch tensor first
parcels = pt.tensor(parcels, dtype=pt.get_default_dtype())
dist = compute_dist(coord, 2, backend='torch')

In [7]:
wcorr_array, bcorr_array, dcbc_array = [],[],[]

tic = time.perf_counter()
for sub in returnsubj:
    data = T['data'][(T['SN'] == sub) & (T['sess'] == session)].T
    data = pt.tensor(data, dtype=pt.get_default_dtype())
    R = compute_DCBC(maxDist=maxDist, func=data, dist=dist, binWidth=binWidth, parcellation=parcels, backend='torch')
    wcorr_array.append(R['corr_within'])
    bcorr_array.append(R['corr_between'])
    dcbc_array.append(R['DCBC'])

toc = time.perf_counter()
print(f"PyTorch GPU version DCBC calculation used {toc - tic:0.4f}s")
# print(wcorr_array, bcorr_array, dcbc_array)

PyTorch GPU version DCBC calculation used 1.1822s


As we can see, the PyTorch GPU version only takes 1.2 second to evaluate the parcellation on all 24 subjects' data, comparing to the Numpy CPU version which takes 90 seconds. Therefore, we highly recommend to use our DCBC toolbox in GPU version, especially if you have large sample size.

Note, the current DCBC GPU version requires the distance matrix / subject's functional data can be loaded into GPU as a whole piece. The data streamer is not available now. Therefore, it is likely you encounter a `CUDA out of memory` error for large tensor. If that happens, please use PyTorch CPU tensor computation by set the cuda global flag to Flase, as `pt.cuda.is_available = lambda : False`

Note, the Apple M1/M2 chip is supported in PyTorch version 2.0 and above. The MAC user who wanted to run DCBC on M1/M2 chip can replace the `set_default_tensor_type()` flag with:

In [ ]:
pt.set_default_device('mps')
pt.set_default_dtype(pt.float32)